In [1]:
## 使用Qwen-coder，对保险客户数据表进行SQL查询

In [2]:
import json
import os
import dashscope
from dashscope.api_entities.dashscope_response import Role
import time
import pandas as pd
import re
# 从环境变量获取 dashscope 的 API Key
api_key = os.environ.get('DASHSCOPE_API_KEY')
dashscope.api_key = api_key

# 封装模型响应函数
def get_response(messages):
    response = dashscope.Generation.call(
        model='qwen-coder-plus',
        messages=messages,
        result_format='message'  # 将输出设置为message形式
    )
    return response

# 从模型响应中提取SQL代码
def get_sql_code(response):
    # 查找```sql和```之间的内容
    pattern = r'```sql(.*?)```'
    match = re.search(pattern, response.output.choices[0].message.content, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        # 如果没有找到```sql标记，尝试查找任何```之间的内容
        pattern = r'```(.*?)```'
        match = re.search(pattern, response.output.choices[0].message.content, re.DOTALL)
        if match:
            return match.group(1).strip()
        else:
            # 如果没有找到任何代码块，返回整个响应
            return response.output.choices[0].message.content

# 得到sql
def get_sql(query):
    start_time = time.time()
    sys_prompt = """我正在编写SQL，以下是数据库中的数据表和字段，请思考：哪些数据表和字段是该SQL需要的，然后编写对应的SQL，如果有多个查询语句，请尝试合并为一个。编写SQL请采用```sql
    """
    user_prompt = f"""-- language: SQL
### Question: {query}
### Input: {create_sql}
### Response:
Here is the SQL query I have generated to answer the question `{query}`:
```sql
"""

    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    response = get_response(messages)
    return response

######## 需要人工设置 ########
save_file = f'sql_result_qwen_coder.xlsx'
qa_file = './qa_list-2.txt'                  # QA测试题
sql_file = './data/create_sql.txt'           # SQL数据表

In [3]:
# 读取 SQL数据表
with open(sql_file, 'r', encoding='utf-8') as file:
    create_sql = file.read()
# 读取 SQL问题列表
with open(qa_file, 'r', encoding='utf-8') as file:
    qa_list = file.read()
qa_list = qa_list.split('=====')

# 保存SQL结果
sql_list = []
markdown_list = []
time_list = []
for qa in qa_list:
    query = qa
    query = query.replace('\n', '')
    print(query)
    start_time = time.time()
    # 请求生成sql
    #content, prompt_len = get_sql(query)
    response = get_sql(query)
    use_time = round(time.time()-start_time, 2)
    time_list.append(use_time)
    print('SQL生成时间：', use_time)
    print('response=', response.output.choices[0].message.content)
    # 提取生成的SQL
    sql = get_sql_code(response)
    print('SQL: {}'.format(sql))
    sql_list.append(sql)

获取所有客户的姓名和联系电话。
SQL生成时间： 2.31
response= 根据您的问题“获取所有客户的姓名和联系电话”，只需要从`CustomerInfo`表中提取`Name`和`PhoneNumber`字段。因此，不需要合并多个查询语句，因为所有所需的信息都在同一个表中。

以下是生成的SQL查询：

```sql
SELECT Name, PhoneNumber
FROM CustomerInfo;
```

这个查询将返回`CustomerInfo`表中所有客户的姓名和联系电话。
SQL: SELECT Name, PhoneNumber
FROM CustomerInfo;
找出所有已婚客户的客户ID和配偶姓名。
SQL生成时间： 7.82
response= To find all married customers and their spouse's names, we need to look at the `CustomerInfo` table for customer details and the `PolicyInfo` table for the relationship information. Specifically, we are interested in policies where the relationship is 'Spouse'. We will also need the `BeneficiaryInfo` table to get the names of the beneficiaries who are spouses.

Here is the SQL query that accomplishes this:

```sql
SELECT 
    ci.CustomerID,
    bi.Name AS SpouseName
FROM 
    CustomerInfo ci
JOIN 
    PolicyInfo pi ON ci.CustomerID = pi.CustomerID
JOIN 
    BeneficiaryInfo bi ON pi.BeneficiaryID = bi.BeneficiaryID
WHERE 
    ci.MaritalStatus = 'Marri

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))

In [ ]:
result = pd.DataFrame(columns=['QA', 'SQL', 'time'])
result['QA'] = qa_list
result['SQL'] = sql_list
result['time'] = time_list
result.to_excel(save_file, index=False)
result